<a href="https://colab.research.google.com/github/Tushar1000x/Image_processing/blob/main/IP_LAB_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import cv2
import numpy as np
import zipfile
import matplotlib.pyplot as plt
from sklearn.cluster import MiniBatchKMeans
from sklearn.metrics import pairwise_distances

# -------------------------------
# Step 0: Unzip the Train Dataset if needed
# -------------------------------
zip_path = 'train_dataset.zip'
extract_dir = '.'
if not os.path.exists("train_dataset"):
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_dir)
    print("Unzipped train_dataset.zip into", extract_dir)
else:
    print("train_dataset folder already exists.")

# -------------------------------
# Parameters and Setup
# -------------------------------
data_dir = 'train_dataset'    # Contains folders: landmark1, landmark2, ..., landmark10
vocab_size = 250               # Vocabulary size for BoVW (tune based on your dataset) HIGH==>captures more fine details
image_size = (2000, 2000)       # Standard resize dimensions

# Initialize SIFT extractor
sift = cv2.SIFT_create()

# -------------------------------
# Visualize SIFT Keypoints on a Sample Image
# -------------------------------
sample_folder = os.path.join(data_dir, 'landmark1')
sample_images = [f for f in os.listdir(sample_folder) if f.lower().endswith('.jpg')]
if sample_images:
    sample_image_path = os.path.join(sample_folder, sample_images[0])
    sample_img = cv2.imread(sample_image_path)
    sample_img = cv2.resize(sample_img, image_size)
    gray_sample = cv2.cvtColor(sample_img, cv2.COLOR_BGR2GRAY)
    keypoints, descriptors = sift.detectAndCompute(gray_sample, None)
    sample_img_with_keypoints = cv2.drawKeypoints(sample_img, keypoints, None,
                                                  flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)

    plt.figure(figsize=(8, 6))
    plt.imshow(cv2.cvtColor(sample_img_with_keypoints, cv2.COLOR_BGR2RGB))
    plt.title("Sample SIFT Keypoints")
    plt.axis('off')
    plt.show()
else:
    print("No sample image found in landmark1.")

# -------------------------------
# Step 1: Collect SIFT Descriptors from All Training Images
# -------------------------------
descriptors_list = []  # Collect descriptors from each image
image_paths = []       # Keep track of each image's path (for later matching)
labels = []            # Corresponding labels (folder names)

for landmark in os.listdir(data_dir):
    landmark_dir = os.path.join(data_dir, landmark)
    if not os.path.isdir(landmark_dir):
        continue
    for filename in os.listdir(landmark_dir):
        if not filename.lower().endswith('.jpg'):
            continue
        img_path = os.path.join(landmark_dir, filename)
        img = cv2.imread(img_path)
        if img is None:
            continue
        img = cv2.resize(img, image_size)
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        kp, des = sift.detectAndCompute(gray, None)
        if des is not None:
            descriptors_list.append(des)
            image_paths.append(img_path)
            labels.append(landmark)

if len(descriptors_list) == 0:
    raise RuntimeError("No SIFT descriptors found; please check your dataset images.")

all_descriptors = np.vstack(descriptors_list)
print("Collected a total of", all_descriptors.shape[0], "descriptors.")

# -------------------------------
# Step 2: Build the Visual Vocabulary using k-means clustering
# -------------------------------
kmeans = MiniBatchKMeans(n_clusters=vocab_size, batch_size=vocab_size * 3, random_state=42)
kmeans.fit(all_descriptors)
print("K-means clustering completed. Vocabulary size:", vocab_size)

# -------------------------------
# Helper Function: Extract BoVW Features with L2 Normalization
# -------------------------------
def extract_bovw_features(image_path, sift, kmeans, image_size):
    """
    Extract a normalized Bag-of-Visual-Words histogram from an image.
    """
    img = cv2.imread(image_path)
    if img is None:
        raise ValueError(f"Could not read image: {image_path}")
    img = cv2.resize(img, image_size)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    kp, des = sift.detectAndCompute(gray, None)
    histogram = np.zeros(kmeans.n_clusters)
    if des is not None:
        words = kmeans.predict(des)
        for w in words:
            histogram[w] += 1
        norm = np.linalg.norm(histogram)
        if norm > 0:
            histogram = histogram / norm
    return histogram

# -------------------------------
# Step 3: Build the Training Database (BoVW Histograms)
# -------------------------------
train_features = []
for path in image_paths:
    hist = extract_bovw_features(path, sift, kmeans, image_size)
    train_features.append(hist)
train_features = np.array(train_features)
train_labels = np.array(labels)
print("Training database built with", len(train_features), "image histograms.")

# Visualize BoVW Histogram for a Sample Image
sample_hist = extract_bovw_features(sample_image_path, sift, kmeans, image_size)
plt.figure(figsize=(10, 4))
plt.bar(range(vocab_size), sample_hist, color='navy')
plt.xlabel("Visual Word Index")
plt.ylabel("Normalized Frequency")
plt.title("BoVW Histogram for Sample Image")
plt.show()

# -------------------------------
# Step 4: Updated Classification Function Returning best_idx
# -------------------------------
def chi2_distance(histA, histB, eps=1e-10):
    return 0.5 * np.sum(((histA - histB) ** 2) / (histA + histB + eps))

def classify_test_image(test_image_path, sift, kmeans, train_features, train_labels, image_size, distance_metric='chi2'):
    """
    Classify a test image by comparing its BoVW histogram to that of the training set.
    Returns the predicted label, the matched image path, the distance array, and the best match index.
    """
    test_hist = extract_bovw_features(test_image_path, sift, kmeans, image_size)

    if distance_metric == 'euclidean':
        distances = pairwise_distances([test_hist], train_features, metric='euclidean')[0]
    elif distance_metric == 'chi2':
        distances = np.array([chi2_distance(test_hist, hist) for hist in train_features])
    else:
        raise ValueError("Unsupported distance metric.")

    best_idx = np.argmin(distances)
    predicted_label = train_labels[best_idx]
    matched_image_path = image_paths[best_idx]
    return predicted_label, matched_image_path, distances, best_idx

# -------------------------------
# Step 5: Visual Evaluation for a Test Image and Feature Matching
# -------------------------------
# Specify the test image path (update this path to point to your actual test image)
test_image_path = 'test.jpg'  # Replace with your test image path

try:
    predicted_label, matched_img_path, distances, best_idx = classify_test_image(
        test_image_path, sift, kmeans, train_features, train_labels, image_size, distance_metric='chi2'
    )
    print(f"Predicted Landmark: {predicted_label}")
    print(f"Closest Matching Training Image: {matched_img_path}")

    # Side-by-Side Visualization: Test Image and Closest Matching Training Image
    test_img = cv2.imread(test_image_path)
    test_img_resized = cv2.resize(test_img, image_size)
    matched_img = cv2.imread(matched_img_path)
    matched_img_resized = cv2.resize(matched_img, image_size)

    plt.figure(figsize=(12, 6))
    plt.subplot(1, 2, 1)
    plt.imshow(cv2.cvtColor(test_img_resized, cv2.COLOR_BGR2RGB))
    plt.title("Test Image")
    plt.axis('off')

    plt.subplot(1, 2, 2)
    plt.imshow(cv2.cvtColor(matched_img_resized, cv2.COLOR_BGR2RGB))
    plt.title(f"Matched Image\nLabel: {predicted_label}")
    plt.axis('off')
    plt.show()

    # Visualize Distance Metrics for the Test Image Evaluation
    plt.figure(figsize=(14, 5))
    indices = np.arange(len(distances))
    plt.bar(indices, distances, color='teal')
    plt.xlabel("Training Image Index")
    plt.ylabel("Distance (chi-square)")
    plt.title("Distance from Test Image to Each Training Image")
    plt.bar(best_idx, distances[best_idx], color='red', label='Best Match')
    plt.legend()
    plt.show()

    # -------------------------------
    # Step 6: Visualizing SIFT Keypoint Matches
    # -------------------------------
    # Extract SIFT keypoints and descriptors for the test image
    test_gray = cv2.cvtColor(test_img_resized, cv2.COLOR_BGR2GRAY)
    kp1, des1 = sift.detectAndCompute(test_gray, None)

    # Extract SIFT keypoints and descriptors for the matched training image
    matched_gray = cv2.cvtColor(matched_img_resized, cv2.COLOR_BGR2GRAY)
    kp2, des2 = sift.detectAndCompute(matched_gray, None)

    # Use BFMatcher to find matches between descriptors
    bf = cv2.BFMatcher(cv2.NORM_L2, crossCheck=True)
    matches = bf.match(des1, des2)
    matches = sorted(matches, key=lambda x: x.distance)  # sort matches by distance

    # Draw the top 30 matches
    img_matches = cv2.drawMatches(test_img_resized, kp1, matched_img_resized, kp2, matches[:30], None, flags=2)

    plt.figure(figsize=(15, 8))
    plt.imshow(cv2.cvtColor(img_matches, cv2.COLOR_BGR2RGB))
    plt.title("SIFT Keypoint Matches Between Test and Matched Training Images")
    plt.axis('off')
    plt.show()

except Exception as e:
    print("Error in test image classification:", e)


In [ ]:
import os
import cv2
import numpy as np
import zipfile
import random
import matplotlib.pyplot as plt
import pandas as pd  # For creating a table
from sklearn.cluster import MiniBatchKMeans
from sklearn.metrics import pairwise_distances
from IPython.display import display, HTML

# -------------------------------
# Step 0: Unzip the Test Dataset if needed
# -------------------------------
test_zip_path = 'test_dataset.zip'
test_extract_dir = '.'
if not os.path.exists("test_dataset"):
    with zipfile.ZipFile(test_zip_path, 'r') as zip_ref:
        zip_ref.extractall(test_extract_dir)
    print("Unzipped test_dataset.zip into", test_extract_dir)
else:
    print("test_dataset folder already exists.")

# -------------------------------
# Function Definitions (Assuming these functions are available from previous code)
# -------------------------------
def extract_bovw_features(image_path, sift, kmeans, image_size):
    """
    Extract a normalized Bag-of-Visual-Words histogram from an image.
    """
    img = cv2.imread(image_path)
    if img is None:
        raise ValueError(f"Could not read image: {image_path}")
    img = cv2.resize(img, image_size)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    kp, des = sift.detectAndCompute(gray, None)
    histogram = np.zeros(kmeans.n_clusters)
    if des is not None:
        words = kmeans.predict(des)
        for w in words:
            histogram[w] += 1
        norm = np.linalg.norm(histogram)
        if norm > 0:
            histogram = histogram / norm
    return histogram

def chi2_distance(histA, histB, eps=1e-10):
    return 0.5 * np.sum(((histA - histB) ** 2) / (histA + histB + eps))

def classify_test_image(test_image_path, sift, kmeans, train_features, train_labels, image_size, distance_metric='chi2'):
    """
    Classify a test image by comparing its BoVW histogram to that of the training set.
    Returns the predicted label, matched training image path, distances array, and best_idx.
    """
    test_hist = extract_bovw_features(test_image_path, sift, kmeans, image_size)

    if distance_metric == 'euclidean':
        distances = pairwise_distances([test_hist], train_features, metric='euclidean')[0]
    elif distance_metric == 'chi2':
        distances = np.array([chi2_distance(test_hist, hist) for hist in train_features])
    else:
        raise ValueError("Unsupported distance metric.")

    best_idx = np.argmin(distances)
    predicted_label = train_labels[best_idx]
    matched_image_path = image_paths[best_idx]
    return predicted_label, matched_image_path, distances, best_idx

# -------------------------------
# Assumptions:
# - The training database and BoVW model have been built.
# - Variables 'sift', 'kmeans', 'train_features', 'train_labels', 'image_paths',
#   and 'image_size' are already defined in the workspace.
# -------------------------------

# -------------------------------
# Step 1: Predict a Random Test Image from Test Set
# -------------------------------
test_dataset_path = "test_dataset"
test_images = []

# Collect all test image paths with their ground truth labels (from folder names)
for landmark in os.listdir(test_dataset_path):
    landmark_dir = os.path.join(test_dataset_path, landmark)
    if not os.path.isdir(landmark_dir):
        continue
    for filename in os.listdir(landmark_dir):
        if filename.lower().endswith(".jpg"):
            full_path = os.path.join(landmark_dir, filename)
            test_images.append((full_path, landmark))  # (image_path, actual_label)

# Pick a random test image for individual visualization
random_test_image_path, true_label = random.choice(test_images)

# Classify the random image
predicted_label, matched_img_path, distances, best_idx = classify_test_image(
    random_test_image_path, sift, kmeans, train_features, train_labels, image_size, distance_metric='chi2'
)

print(f"\n🎯 RANDOM TEST IMAGE PREDICTION")
print(f"Test Image Path: {random_test_image_path}")
print(f"Actual Label (Folder): {true_label}")
print(f"Predicted Label       : {predicted_label}")
print(f"Matched Train Image   : {matched_img_path}")

# -------------------------------
# Optional: Visualize Matches and Distance Plot for the Random Test Image
# -------------------------------
# Read and resize images
test_img = cv2.imread(random_test_image_path)
test_img = cv2.resize(test_img, image_size)
matched_img = cv2.imread(matched_img_path)
matched_img = cv2.resize(matched_img, image_size)

# Get SIFT keypoints for both images
kp1, des1 = sift.detectAndCompute(cv2.cvtColor(test_img, cv2.COLOR_BGR2GRAY), None)
kp2, des2 = sift.detectAndCompute(cv2.cvtColor(matched_img, cv2.COLOR_BGR2GRAY), None)

# Create BFMatcher and compute matches
bf = cv2.BFMatcher(cv2.NORM_L2, crossCheck=True)
matches = bf.match(des1, des2)
matches = sorted(matches, key=lambda x: x.distance)

# Draw the top 30 matches
img_matches = cv2.drawMatches(test_img, kp1, matched_img, kp2, matches[:30], None, flags=2)
plt.figure(figsize=(15, 7))
plt.imshow(cv2.cvtColor(img_matches, cv2.COLOR_BGR2RGB))
plt.title("Top 30 SIFT Keypoint Matches")
plt.axis('off')
plt.show()

# Plot the distance bar chart for the random test image
plt.figure(figsize=(12, 4))
indices = np.arange(len(distances))
plt.bar(indices, distances, color='teal')
plt.xlabel("Training Image Index")
plt.ylabel("Chi-square Distance")
plt.title("Test Image Distance to All Training Images")
plt.bar(best_idx, distances[best_idx], color='red', label='Best Match')
plt.legend()
plt.show()

# -------------------------------
# Step 2: Evaluate Accuracy Over Entire Test Dataset and Record Predictions
# -------------------------------
results = []  # List to record (test_image_path, actual_label, predicted_label)
correct = 0
total = 0

print(f"\n📊 Evaluating Accuracy Over {len(test_images)} Test Images...\n")
for test_path, actual_label in test_images:
    pred_label, _, _, _ = classify_test_image(test_path, sift, kmeans, train_features, train_labels, image_size)
    results.append((test_path, actual_label, pred_label))
    if pred_label == actual_label:
        correct += 1
    total += 1

accuracy = 100 * correct / total
print(f"✅ Overall Accuracy: {accuracy:.2f}%  ({correct}/{total} correct predictions)")

# -------------------------------
# Step 3: Convert the Results List into a Table and Print
# ------------------------------

# Assuming 'results' is a list of tuples: (Image Path, Actual Label, Predicted Label)
results_df = pd.DataFrame(results, columns=["Image Path", "Actual Label", "Predicted Label"])

# Create a matplotlib figure and axis
fig, ax = plt.subplots(figsize=(24, 10))  # Adjust height based on number of rows

# Hide axes
ax.axis('tight')
ax.axis('off')

# Create the table
table = ax.table(cellText=results_df.values,
                 colLabels=results_df.columns,
                 cellLoc='center',
                 loc='center')

# # Customize table style
# table.auto_set_font_size(False)
# table.set_fontsize(12)
table.scale(1, 4)  # scale horizontally and vertically

# Change header row properties
for (row, col), cell in table.get_celld().items():
    if row == 0:  # header row
        cell.set_text_props(weight='bold', color='white')
        cell.set_facecolor('#4CAF50')  # green header background
    else:
        # Alternate background color (optional)
        if row % 2 == 0:
            cell.set_facecolor('#f1f1f2')
        else:
            cell.set_facecolor('white')

plt.title("Test Image Predictions", fontsize=16, pad=20)
plt.show()

